In [1]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
import numpy as np
import pymysql

In [2]:
#conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='101Monkeybob!',db='Final_Project')
#df=pd.read_sql_query("SELECT * FROM 'house' ",conn)


In [3]:
# Import Clean House Data
house_data = pd.read_csv("Raw Data/clean_house_data.csv",low_memory = False)
PRIME_data = pd.read_csv('Raw Data/PRIME_data.csv', low_memory = False)
imp_df = pd.DataFrame(house_data)
prime_df = pd.DataFrame(PRIME_data)
imp_df.head()

,BATHRM,HF_BATHRM,AC,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,LONGITUDE,LATITUDE,QUADRANT,PRICE,ZIPCODE,WARD,PRIME
0,3,1,Y,9,5,2016.0,2.0,4,1680,-77.040678,38.914684,NW,2100000.0,20009.0,2,3.750
1,3,1,Y,8,5,2006.0,2.0,3,1680,-77.040629,38.914683,NW,1602000.0,20009.0,2,7.875
2,3,2,Y,10,5,2010.0,1.0,4,2196,-77.039715,38.914331,NW,1950000.0,20009.0,2,3.250
3,3,1,Y,8,4,2011.0,2.0,1,1627,-77.040129,38.915408,NW,1050000.0,20009.0,2,3.250
4,3,1,Y,7,3,2018.0,2.0,1,1424,-77.039903,38.915017,NW,1430000.0,20009.0,2,5.125


In [4]:
imp_df = imp_df.drop(columns=['LONGITUDE','LATITUDE'])
imp_df.dtypes

BATHRM          int64
HF_BATHRM       int64
AC             object
ROOMS           int64
BEDRM           int64
SALEDATE      float64
KITCHENS      float64
FIREPLACES      int64
LANDAREA        int64
QUADRANT       object
PRICE         float64
ZIPCODE       float64
WARD            int64
PRIME         float64
dtype: object

In [5]:
imp_cat = imp_df.dtypes[imp_df.dtypes == "object"].index.tolist()


In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(imp_df[imp_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(imp_cat)
encode_df.head()

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,AC_0,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [7]:
encode_df = encode_df.drop(columns = 'AC_0')
# Merge one-hot encoded features and drop the originals
imp_df = imp_df.merge(encode_df,left_index=True, right_index=True)
imp_df = imp_df.drop(imp_cat,1)
imp_df.head()

C:\Users\FrantheMan\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
0,3,1,9,5,2016.0,2.0,4,1680,2100000.0,20009.0,2,3.750,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1,8,5,2006.0,2.0,3,1680,1602000.0,20009.0,2,7.875,0.0,1.0,0.0,1.0,0.0,0.0
2,3,2,10,5,2010.0,1.0,4,2196,1950000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
3,3,1,8,4,2011.0,2.0,1,1627,1050000.0,20009.0,2,3.250,0.0,1.0,0.0,1.0,0.0,0.0
4,3,1,7,3,2018.0,2.0,1,1424,1430000.0,20009.0,2,5.125,0.0,1.0,0.0,1.0,0.0,0.0


In [8]:
index = imp_df[imp_df.PRICE >= 1500000].index
index2 = imp_df[imp_df.PRICE <= 25000].index
imp_df = imp_df.drop(index)
imp_df = imp_df.drop(index2)


In [9]:
print(imp_df.PRICE.max())
print(imp_df.PRICE.min())


1499500.0
25007.0


In [40]:
price_df = imp_df.PRICE
price_df.to_csv('Raw Data/price.csv', index = False)

In [10]:
#Binning Price data
bins = [25000, 50000, 75000,100000, 125000, 150000, 175000, 200000, 225000,250000,
        275000,300000,325000,350000,375000,400000,425000,450000,475000,500000,
        525000,550000,575000,600000,625000,650000,675000,700000,725000,750000,
        775000, 800000,825000,850000,875000,900000,925000,950000,975000,1000000,
        1025000,1050000,1075000,1100000,1125000,1150000,1175000,1200000,1225000,1250000,
        1275000,1300000,1325000,1350000,1375000,1400000,1425000, 1450000,1475000,1500000]
labels = [x*1 for x in range(1,60)]
cut = pd.cut(imp_df['PRICE'], bins=bins, labels=labels)
cut_df = pd.DataFrame(cut)
cut_df['REAL_PRICE'] = imp_df.PRICE
cut_df = cut_df.rename(columns={'PRICE':'INDEX_BIN'})
cut_df.head(50)

,INDEX_BIN,REAL_PRICE
3,41,1050000.0
4,57,1430000.0
5,52,1325000.0
6,49,1240000.0
7,23,592250.0
8,1,33232.0
9,36,907400.0
10,42,1065000.0
11,43,1100000.0
13,48,1210000.0


In [11]:
imp_df = pd.merge(imp_df, cut_df, left_index=True, right_index=True)
imp_df.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,PRICE,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN,REAL_PRICE
3,3,1,8,4,2011.0,2.0,1,1627,1050000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,41,1050000.0
4,3,1,7,3,2018.0,2.0,1,1424,1430000.0,20009.0,2,5.125000,0.0,1.0,0.0,1.0,0.0,0.0,57,1430000.0
5,3,1,5,3,2011.0,2.0,1,1815,1325000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,52,1325000.0
6,3,1,8,3,2010.0,1.0,0,1424,1240000.0,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,49,1240000.0
7,3,1,9,3,2001.0,2.0,1,1424,592250.0,20009.0,2,6.772727,0.0,1.0,0.0,1.0,0.0,0.0,23,592250.0


In [12]:
imp_df = imp_df.astype({'INDEX_BIN': 'int32'})

In [ ]:
# Dropping rows with a saledate less than 2000
new_df = imp_df[imp_df.SALEDATE >= 2000]
new_df.head(50)

In [35]:
new_df.to_csv("Raw Data/clean_house_data.csv", index = False)

In [14]:
#remove_n = 15000
#drop_indices = np.random.choice(new_df.index, remove_n, replace=False)
#new_df = new_df.drop(drop_indices)
#new_df

In [15]:
new_df = new_df.drop(columns=['REAL_PRICE','PRICE'])
new_df

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW,INDEX_BIN
3,3,1,8,4,2011.0,2.0,1,1627,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,41
4,3,1,7,3,2018.0,2.0,1,1424,20009.0,2,5.125000,0.0,1.0,0.0,1.0,0.0,0.0,57
5,3,1,5,3,2011.0,2.0,1,1815,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,52
6,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0,49
7,3,1,9,3,2001.0,2.0,1,1424,20009.0,2,6.772727,0.0,1.0,0.0,1.0,0.0,0.0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55023,2,0,10,4,2004.0,2.0,0,5837,20032.0,8,4.750000,1.0,0.0,0.0,0.0,0.0,1.0,3
55024,2,0,10,4,2016.0,2.0,0,5302,20032.0,8,3.750000,0.0,1.0,0.0,0.0,0.0,1.0,7
55025,2,0,10,5,2012.0,2.0,0,5348,20032.0,8,3.250000,1.0,0.0,0.0,0.0,0.0,1.0,3
55026,2,0,10,4,2017.0,2.0,0,3046,20032.0,8,4.250000,1.0,0.0,0.0,0.0,0.0,1.0,8


In [16]:
# Define the features 
X = new_df.copy()
X = X.drop("INDEX_BIN", axis = 1)
X.head()

,BATHRM,HF_BATHRM,ROOMS,BEDRM,SALEDATE,KITCHENS,FIREPLACES,LANDAREA,ZIPCODE,WARD,PRIME,AC_N,AC_Y,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
3,3,1,8,4,2011.0,2.0,1,1627,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0
4,3,1,7,3,2018.0,2.0,1,1424,20009.0,2,5.125000,0.0,1.0,0.0,1.0,0.0,0.0
5,3,1,5,3,2011.0,2.0,1,1815,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0
6,3,1,8,3,2010.0,1.0,0,1424,20009.0,2,3.250000,0.0,1.0,0.0,1.0,0.0,0.0
7,3,1,9,3,2001.0,2.0,1,1424,20009.0,2,6.772727,0.0,1.0,0.0,1.0,0.0,0.0


In [17]:
# Define target set
y = new_df['INDEX_BIN'].ravel()
y[:5]

array([41, 57, 52, 49, 23])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators = 100, random_state = 2)
model.fit(X_train,y_train)

BalancedRandomForestClassifier(random_state=2)

In [20]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
accuracy_score = balanced_accuracy_score(y_test, y_pred)
accuracy_score

0.09283437355870366

In [21]:
#Fitting the model
confusion_matrix(y_test,y_pred)

array([[ 9, 11,  9, ...,  0,  0,  0],
       [20, 25, 11, ...,  0,  0,  0],
       [32, 41, 48, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  4,  3,  1],
       [ 0,  0,  0, ...,  3,  2,  0],
       [ 0,  0,  0, ...,  2,  5,  2]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.03      0.18      0.97      0.05      0.41      0.16        51
          2       0.09      0.22      0.98      0.13      0.46      0.20       115
          3       0.21      0.21      0.98      0.21      0.46      0.19       225
          4       0.26      0.20      0.98      0.22      0.44      0.18       303
          5       0.28      0.21      0.98      0.24      0.45      0.19       388
          6       0.20      0.17      0.98      0.18      0.41      0.15       369
          7       0.20      0.21      0.97      0.20      0.45      0.19       392
          8       0.15      0.12      0.98      0.14      0.35      0.11       369
          9       0.17      0.13      0.97      0.15      0.36      0.12       419
         10       0.18      0.19      0.97      0.19      0.43      0.17       379
         11       0.24      0.19      0.98      0.21      0.43      0.17       407
   

In [23]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.21372415153319227, 'LANDAREA'),
 (0.13444560494477156, 'SALEDATE'),
 (0.1088131821770803, 'ROOMS'),
 (0.090220832900863, 'PRIME'),
 (0.08141769739575315, 'ZIPCODE'),
 (0.07237805352478296, 'BEDRM'),
 (0.05830858282102541, 'BATHRM'),
 (0.05572179043953314, 'HF_BATHRM'),
 (0.051126472466804, 'WARD'),
 (0.05003256547313031, 'FIREPLACES'),
 (0.026771331367504453, 'KITCHENS'),
 (0.013430343646713921, 'AC_Y'),
 (0.01336879257635262, 'AC_N'),
 (0.011264250521002012, 'QUADRANT_NE'),
 (0.009425038146733103, 'QUADRANT_SE'),
 (0.007914512611167188, 'QUADRANT_NW'),
 (0.0016367974535906544, 'QUADRANT_SW')]